<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Best_attention_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required packages 

import os
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from random import randrange 
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'

--2022-05-08 13:53:26--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.200.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  75.9MB/s    in 27s     

2022-05-08 13:53:54 (70.6 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
# Paths of train and valid datasets.
train_data_path = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data_path = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"

# Saving the files in list
with open(train_data_path, "r", encoding="utf-8") as file:
    train_data_lines = file.read().split("\n")

with open(val_data_path, "r", encoding="utf-8") as file:
    val_data_lines = file.read().split("\n")

# popping the empty character of the lists 
val_data_lines.pop()
train_data_lines.pop()

# Fixed parameter
batch_size = 64

['கோவேறு\tkovaeru\t1', 'இதையெல்லாம்\tidhaiyellaam\t2']


In [ ]:
# embedding train data

def embed_train_data(train_data_lines):

    lenk = len(train_data_lines) - 1
    train_input_data = []
    train_target_data = []
    input_data_characters = set()
    target_data_characters = set()
    
    for line in train_data_lines[: lenk]:
        target_data, input_data, _ = line.split("\t")

        # We are using "tab" as the "start sequence" and "\n" as "end sequence".
        target_data = "\t" + target_data + "\n"
        train_input_data.append(input_data)
        train_target_data.append(target_data)

        # Finding unique characters.
        for ch in input_data:
            if ch not in input_data_characters:
                input_data_characters.add(ch)
        for ch in target_data:
            if ch not in target_data_characters:
                target_data_characters.add(ch)

    print("Number of samples:", len(train_input_data))
    # adding space 
    input_data_characters.add(" ")
    target_data_characters.add(" ")

    # sorting
    input_data_characters = sorted(list(input_data_characters))
    target_data_characters = sorted(list(target_data_characters))

    # maximum length of the words
    encoder_max_length = max([len(txt) for txt in train_input_data])
    decoder_max_length = max([len(txt) for txt in train_target_data])

    print("Max length for inputs:", encoder_max_length)
    print("Max length for outputs:", decoder_max_length)

    # number of input and target characters
    num_encoder_tokens = len(input_data_characters)
    num_decoder_tokens = len(target_data_characters)  
    
    print("unique input tokens:", num_encoder_tokens)
    print("output tokens:", num_decoder_tokens)

    # create an index
    input_token_idx = dict([(char, i) for i, char in enumerate(input_data_characters)])
    target_token_idx = dict([(char, i) for i, char in enumerate(target_data_characters)])
   
    # creating 0 array for encoder,decoder 
    encoder_input_data = np.zeros((len(train_input_data), encoder_max_length), dtype="float32")

    decoder_input_data = np.zeros((len(train_input_data), decoder_max_length), dtype="float32")

    decoder_target_data = np.zeros((len(train_input_data), decoder_max_length, num_decoder_tokens), dtype="float32")

    # index of the character is encoded for all the sample whereas target data is one hot encoded.
    for i, (input_data, target_data) in enumerate(zip(train_input_data, train_target_data)):
        for t, char in enumerate(input_data):
            encoder_input_data[i, t] = input_token_idx[char]
        
        encoder_input_data[i, t + 1:] = input_token_idx[" "]
        
        # decoder data
        for t, char in enumerate(target_data):
            # decoder_target_data is one timestep ahead of decoder_input_data
            decoder_input_data[i, t] = target_token_idx[char]

            if t > 0:
                # excluding the start character since decoder target data is one timestep ahead.
                decoder_target_data[i, t - 1, target_token_idx[char]] = 1.0
        # append the remaining positions with empty space
       
        decoder_input_data[i, t + 1:] = target_token_idx[" "]
        decoder_target_data[i, t:, target_token_idx[" "]] = 1.0

    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_idx,target_token_idx,encoder_max_length,decoder_max_length

In [ ]:
# embedding validation data

def embed_val_data(val_data_lines,num_decoder_tokens,input_token_idx,target_token_idx):
    val_input_data = []
    val_target_data = []
    lenk = len(val_data_lines) - 1

    for line in val_data_lines[: lenk]:
        target_data, input_data, _ = line.split("\t")
        
        # We use "tab" as the "start sequence" character and "\n" as "end sequence" character.
        target_data = "\t" + target_data + "\n"
        val_input_data.append(input_data)
        val_target_data.append(target_data)

    val_encoder_max_length = max([len(txt) for txt in val_input_data])
    val_decoder_max_length = max([len(txt) for txt in val_target_data])

    val_encoder_input_data = np.zeros((len(val_input_data), val_encoder_max_length), dtype="float32")
    val_decoder_input_data = np.zeros((len(val_input_data), val_decoder_max_length), dtype="float32")
    val_decoder_target_data = np.zeros((len(val_input_data), val_decoder_max_length, num_decoder_tokens), dtype="float32")

    for i, (input_data, target_data) in enumerate(zip(val_input_data, val_target_data)):
        for t, ch in enumerate(input_data):
            val_encoder_input_data[i, t] = input_token_idx[ch]
        val_encoder_input_data[i, t + 1:] = input_token_idx[" "]
        
        for t, ch in enumerate(target_data):
            # decoder_target_data is one timestep ahead of decoder_input_data
            val_decoder_input_data[i, t] = target_token_idx[ch]
            if t > 0:
                # excluding the start character since decoder target data is one timestep ahead.
                val_decoder_target_data[i, t - 1, target_token_idx[ch]] = 1.0
       
        val_decoder_input_data[i, t + 1:] = target_token_idx[" "]
        val_decoder_target_data[i, t:, target_token_idx[" "]] = 1.0

    return val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_idx,val_target_data

In [ ]:
# Embedding data
encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_idx,target_token_idx,encoder_max_length,decoder_max_length = embed_train_data(train_data_lines)

val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_idx,val_target_data = embed_val_data(val_data_lines,num_decoder_tokens,input_token_idx,target_token_idx)

reverse_input_char_index = dict((i, char) for char, i in input_token_idx.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_idx.items())

In [ ]:
#Attention Layer
import tensorflow as tf

class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    
    Credits to Tensorflow.org and https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.        

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)
            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            #print("Started context")
            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]
        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim
        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )
        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )
        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
# RNN model 
import tensorflow as tf
def seq2seq(embedding_size, n_encoder_tokens, n_decoder_tokens, n_encoder_layers,n_decoder_layers, latent_dimension, cell_type,
            target_token_idx, decoder_max_length, reverse_target_char_index,dropout,encoder_input_data, decoder_input_data,
            decoder_target_data,batch_size,epochs
):
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')

  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  e_layer=n_encoder_layers

  # RNN 

  if cell_type=="RNN":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)

    encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)

    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
        
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax",name="dense_1")
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])                

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=WandbCallback()
      )
    model.summary()

    # Inference model
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs, state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs


    attention_layer = model.get_layer('attention_layer')
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs])

    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs, attention_out])
    decoder_dense = model.get_layer("dense_1")
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h],[decoder_dense_outputs]+decoder_states+[attention_states])
    return encoder_model, decoder_model

  # GRU

  elif cell_type=="GRU":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)
   
    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
    
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='dense_1')
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])#, metrics=[my_metric]                 

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=WandbCallback()
      )
    model.summary()

    # Inference model
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs,state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")


    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs

    attention_layer = model.get_layer('attention_layer')
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs])
    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs, attention_out])

    decoder_dense = model.get_layer('dense_1')
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h],[decoder_dense_outputs]+decoder_states+[attention_states])

    return encoder_model, decoder_model

  # LSTM

  elif cell_type=="LSTM":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h, state_c = encoder(embed)
  
    encoder_states = None
    encoder_states = [state_h, state_c]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,name='decoder_hidden_1', dropout=dropout)
    
    # initial state of decoder is encoder's last state of last layer
    decoder_outputs, _,_ = decoder(embed_dec, initial_state=encoder_states)
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='dense_1')
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['accuracy'])#, metrics=[my_metric]                 
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=WandbCallback()
      )
    model.summary()

    # Inference model
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc, state_c_enc = model.get_layer('encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs,state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")

    for j in range(1,n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        decoder_state_input_c = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs_inf, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec, state_c_dec]
        decoder_states_inputs += current_states_inputs

    attention_layer = model.get_layer('attention_layer')
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs_inf])
    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs_inf, attention_out])

    decoder_dense = model.get_layer('dense_1')
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h,decoder_state_input_c],[decoder_dense_outputs]+decoder_states+[attention_states])

    return encoder_model, decoder_model

In [ ]:
def decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension):
            # Encode the input as state vectors.
            if cell_type=='LSTM':
              encoder_first_outputs,state_h,state_c= encoder_model.predict(input_seq)
              states_value=[state_h,state_c]
            else:
              encoder_first_outputs,states_value= encoder_model.predict(input_seq)

            # Generate empty target sequence of length 1.
            empty_seq = np.zeros((1, 1))
            empty_seq[0, 0] = target_token_index["\t"]
            target_seq = empty_seq

            # Sampling loop for a batch of sequences
            # (to simplify, here we assume a batch of size 1).
            stop_condition = False
            decoded_sentence = ""
            attention_weights = []
            while not stop_condition:
                if cell_type == "LSTM":
                    output_tokens, h, c,attn = decoder_model.predict([target_seq, encoder_first_outputs] + states_value)
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = states_value[0].reshape((1, latent_dimension))
                    output_tokens, h ,attn= decoder_model.predict([target_seq] + [encoder_first_outputs] + [states_value])
                # dec_ind = np.argmax(output_tokens, axis=-1)[0, 0]
                attention_weights.append(attn)
                # Sample a token
                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_char = reverse_target_char_index[sampled_token_index]
                decoded_sentence += sampled_char

                # Exit condition: either hit max length
                # or find stop character.
                if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
                    stop_condition = True

                # Update the target sequence (of length 1).
                target_seq = np.zeros((1, 1))
                target_seq[0, 0]= sampled_token_index

                # Update states
                if cell_type == "LSTM":
                    states_value = [h, c]
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = [h]
            return decoded_sentence, attention_weights 

In [ ]:
def accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension, verbose=False):
        n_correct = 0
        n_total = 0
        for seq_index in range(len(val_encoder_input_data)):
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            decoded_sentence,attention_weights = decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension)

            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1

            n_total += 1
            print(n_correct)

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())

        return n_correct * 100.0 / n_total,attention_weights

In [ ]:
# parameters
embedding_size=128
n_encoder_tokens=num_encoder_tokens
n_decoder_tokens=num_decoder_tokens
n_encoder_layers=1
n_decoder_layers=1
latent_dimension=256
cell_type='LSTM'
epochs=10
target_token_index=target_token_index
max_decoder_seq_length=max_decoder_seq_length
reverse_target_char_index=reverse_target_char_index
dropout=0.2

In [ ]:
# Best model
encoder_model, decoder_model=seq2seq(embedding_size, n_encoder_tokens,n_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)

Epoch 1/10
1066/1066 [==============================] - 95s 81ms/step - loss: 0.6173 - accuracy: 0.8250
Epoch 2/10
1066/1066 [==============================] - 86s 81ms/step - loss: 0.1670 - accuracy: 0.9541
Epoch 3/10
1066/1066 [==============================] - 86s 81ms/step - loss: 0.1078 - accuracy: 0.9704
Epoch 4/10
1066/1066 [==============================] - 86s 81ms/step - loss: 0.0893 - accuracy: 0.9753
Epoch 5/10
1066/1066 [==============================] - 87s 81ms/step - loss: 0.0784 - accuracy: 0.9782
Epoch 6/10
1066/1066 [==============================] - 86s 81ms/step - loss: 0.0705 - accuracy: 0.9803
Epoch 7/10
1066/1066 [==============================] - 85s 80ms/step - loss: 0.0641 - accuracy: 0.9819
Epoch 8/10
1066/1066 [==============================] - 86s 80ms/step - loss: 0.0589 - accuracy: 0.9834
Epoch 9/10
1066/1066 [==============================] - 86s 80ms/step - loss: 0.0547 - accuracy: 0.9846
Epoch 10/10
1066/1066 [==============================] - 85s 79m

In [ ]:
# compute test accuracy
print('Reading test data')
test_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv"
# open and save the files to lists
with open(test_data, "r", encoding="utf-8") as f:
    test_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
test_lines.pop()
# embedding test
# for test data, almost same
test_input_texts = []
test_target_texts = []
for line in test_lines[: (len(test_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    test_input_texts.append(input_text)
    test_target_texts.append(target_text)
test_max_encoder_seq_length = max([len(txt) for txt in test_input_texts])
test_max_decoder_seq_length = max([len(txt) for txt in test_target_texts])
test_encoder_input_data = np.zeros(
    (len(test_input_texts), test_max_encoder_seq_length), dtype="float32"
)
test_decoder_input_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length), dtype="float32"
)
test_decoder_target_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1:] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Reading test data


In [ ]:
subset = 0
test_accuracy = accuracy(test_encoder_input_data[0:subset], test_target_texts[0:subset],n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts[0:subset]) if subset>0 \
    else accuracy(test_encoder_input_data, test_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts)
print('Test accuracy: ', test_accuracy)

Test accuracy:  53.16916800233134


<Figure size 720x720 with 0 Axes>

Q6 Connectivity

In [ ]:
import numpy as np
from IPython.display import HTML
from IPython.display import display
import ast

def cstr(s, color = 'black'):
    return "<text style=color:#000;padding-top:1.5px;padding-bottom:1.5px;padding-left:2.5px;padding-right:2.5px;background-color:{}>{} </text>".format(color, s)

def get_clr(value, mode):
        colors = ['#ffffff', '#ecf7fb', '#daeff7', '#c7e7f3', '#b5dfef', '#a2d7eb', '#90cfe7', '#7dc7e3', '#6abfdf', '#58b7db', '#46afd7']
        value = int((value * 100) / 10)
        return colors[value]

def visualize(dec_char, text_colours):
    if (dec_char == "<e>"):
      display(HTML(''.join([cstr(ti, color = ci) for ti, ci in text_colours]) + " <b> &emsp; &lt; e &gt; </b>  &emsp; &nbsp; "))
    else:
      display(HTML(''.join([cstr(ti, color = ci) for ti, ci in text_colours]) + " <b> &emsp; {}</b>  &emsp; &emsp; ".format(dec_char)))
      
def softmax(x):
    denom = sum([np.exp(p) for p in x])
    return [np.exp(p) / denom for p in x]


In [ ]:
def decode_conn(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension,input_word):
            if cell_type=='LSTM':
              encoder_first_outputs,state_h,state_c= encoder_model.predict(input_seq)
              states_value=[state_h,state_c]
            else:
              encoder_first_outputs,states_value= encoder_model.predict(input_seq)
            empty_seq = np.zeros((1, 1))
            empty_seq[0, 0] = target_token_index["\t"]
            target_seq = empty_seq

            stop_condition = False
            decoded_sentence = ""
            attention_weights = []
            
            while not stop_condition:
                f_s=[]
                if cell_type == "LSTM":
                    output_tokens, h, c,attn = decoder_model.predict([target_seq, encoder_first_outputs] + states_value)
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = states_value[0].reshape((1, latent_dimension))
                    output_tokens, h ,attn= decoder_model.predict([target_seq] + [encoder_first_outputs] + [states_value])
                dec_ind = np.argmax(output_tokens, axis=-1)[0, 0]
                #attention_weights.append((dec_ind, attn))

                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_char = reverse_target_char_index[sampled_token_index]
                decoded_sentence += sampled_char

                #print(sampled_char)
                #print(input_word[0][0])
                probs=attn.reshape(-1)
                for p in range(len(input_word[0])):
                  f_s.append(probs[p])
                #print(f_s)
                soft=softmax(f_s)
                # dummy=soft
                # anandh=np.argmax(dummy)
                # for i in range(len(soft)):
                #   if i!=anandh:
                #     soft[i]=0
                #   else:
                #     soft[i]=1
                #print(soft)
                #true_word_array='faarm'
                text_colours=[]
                for prob_index in range(len(input_word[0])) :
                        p = float(soft[prob_index])
                        true_char = input_word[0][prob_index]
                        text= (true_char, get_clr(p, 'c') )
                        text_colours.append(text)
                if sampled_char!='\n':
                  visualize(sampled_char, text_colours)
                else:
                  visualize('<e>', text_colours)

                if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
                    stop_condition = True

                # Update the target sequence (of length 1).
                target_seq = np.zeros((1, 1))
                target_seq[0, 0]= sampled_token_index
                # Update states
                if cell_type == "LSTM":
                    states_value = [h, c]
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = [h]
            print('\n')
            return decoded_sentence

In [ ]:
def accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts, verbose=False):
        n_correct = 0
        n_total = 0
        for seq_index in range(len(val_encoder_input_data)):
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            input_word=test_input_texts[seq_index: seq_index + 1]
            decoded_sentence = decode_conn(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension,input_word)

            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1

            n_total += 1
            #print(input_seq)

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())

        return n_correct * 100.0 / n_total,attention_weights

In [ ]:
subset=20
print(test_input_texts[0:1])
test_accuracy,attention_weights = accuracy(test_encoder_input_data[0:subset], test_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts[0:subset])
print('test_accuracy: ', test_accuracy)

['faarm']




test_accuracy:  25.0


Attention Heat maps


In [ ]:
def accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts ,verbose=False):
        n_correct = 0
        n_total = 0
        index=1
        queries = []
        outputs = []
        ground_truth=[]
        attentions = []
        fig = plt.figure(figsize=(10,10))
        for seq_index in range(len(val_encoder_input_data)):
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            decoded_sentence,att = decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension)
            #print(att)
            # print("Done 1")
            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1  

            n_total += 1
            #print(n_correct)

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())
            
            #print(att)
            attention=np.array(att)
            # print(input_seq)
            # print(attention.reshape(attention.shape[0],attention.shape[-1])[:len(decoded_sentence),:])
            # print("1")
            # print(attention.shape)
            # print(attention.shape[-1])
            plt.subplot(3,3,index)
            plt.imshow(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_sentence)],cmap="viridis")

            plt.xticks(range(attention.shape[0]),decoded_sentence,fontproperties=FontProperties(fname = 'Nirmala.ttf'))
            plt.yticks(range(attention.shape[0]),test_input_texts[seq_index])

            inputs.append(test_input_texts[seq_index])
            outputs.append(decoded_sentence.strip())
            attentions.append(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_sentence)])
            ground_truth.append(val_target_texts[seq_index].strip()) # To remove the tab and newline characters
            index+=1
        #plt.show()
        df_train = pd.DataFrame({"Input": inputs, "Ground Truth" : ground_truth, "Model output":outputs})
        #print(df_train)
        df_train.to_csv('predictions_attention.csv', index=False)
        return n_correct * 100.0 / n_total

In [ ]:
subset = 9
test_accuracy = accuracy(test_encoder_input_data[0:subset], test_target_texts[0:subset],n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts[0:subset]) if subset>0 \
    else accuracy(test_encoder_input_data, test_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts)
print('Test accuracy: ', test_accuracy)